In [1]:
import json
import pandas as pd

with open('/content/input.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
75,fact-28,[What do I do if I'm worried about my mental h...,[The most important thing is to talk to someon...
76,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
77,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
78,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


In [2]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

df = pd.DataFrame.from_dict(dic)
df

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...
...,...,...,...
227,fact-29,How do I know if I'm unwell?,"[If your beliefs , thoughts , feelings or beha..."
228,fact-30,How can I maintain social connections? What if...,"[A lot of people are alone right now, but we d..."
229,fact-31,What's the difference between anxiety and stress?,[Stress and anxiety are often used interchange...
230,fact-32,What's the difference between sadness and depr...,"[Sadness is a normal reaction to a loss, disap..."


In [3]:
df['tag'].unique()

array(['greeting', 'morning', 'afternoon', 'evening', 'night', 'goodbye',
       'thanks', 'no-response', 'neutral-response', 'about', 'skill',
       'creation', 'name', 'help', 'sad', 'stressed', 'worthless',
       'depressed', 'happy', 'casual', 'anxious', 'not-talking', 'sleep',
       'scared', 'death', 'understand', 'done', 'suicide', 'hate-you',
       'hate-me', 'default', 'jokes', 'repeat', 'wrong', 'stupid',
       'location', 'something-else', 'friends', 'ask', 'problem',
       'no-approach', 'learn-more', 'user-agree', 'meditation',
       'user-meditation', 'pandora-useful', 'user-advice',
       'learn-mental-health', 'mental-health-fact', 'fact-1', 'fact-2',
       'fact-3', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9',
       'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15',
       'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-20', 'fact-21',
       'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27',
       'fact-28', 'fact-29', '

In [4]:
import plotly.graph_objects as go

intent_counts = df['tag'].value_counts()
fig = go.Figure(data=[go.Bar(x=intent_counts.index, y=intent_counts.values)])
fig.update_layout(title='Distribution of Intents', xaxis_title='Intents', yaxis_title='Count')
fig.show()

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import plotly.graph_objects as go

In [6]:
X = df['patterns']
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = SVC()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

report = {label: {metric: report[label][metric] for metric in report[label]} for label in report if isinstance(report[label], dict)}

labels = list(report.keys())
evaluation_metrics = ['precision', 'recall', 'f1-score']
metric_scores = {metric: [report[label][metric] for label in labels if label in report] for metric in evaluation_metrics}

fig = go.Figure()
for metric in evaluation_metrics:
    fig.add_trace(go.Bar(name=metric, x=labels, y=metric_scores[metric]))

# fig.update_layout(title='Intent Prediction Model Performance',
#                   xaxis_title='Intent',
#                   yaxis_title='Score',
#                   barmode='group')

# fig.show()


In [9]:
import random

# Function to predict the intent
def predict_intent(user_input):
    user_input_vec = vectorizer.transform([user_input])
    intent = model.predict(user_input_vec)[0]
    return intent

# Function to generate a random response based on the intent
def generate_response(intent):
    print(intent)  # For debugging purposes, to see the intent detected

    # Filter the DataFrame to get the responses for the given intent
    responses = df[df['tag'] == intent]['responses'].values[0]

    # Select a random response
    response = random.choice(responses)
    return response

while True:
    user_input = input("Prompt (press 'q' to quit): ")
    if user_input.lower() == 'q':
        break
    else:
        intent = predict_intent(user_input)
        response = generate_response(intent)
        print("AI:", response)


Prompt (press 'q' to quit): hi
greeting
AI: Hi there. How are you feeling today?
Prompt (press 'q' to quit): because my friends don't like me
greeting
AI: Hi there. How are you feeling today?
Prompt (press 'q' to quit): i'm sad
sad
AI: How long have you been feeling this way?
Prompt (press 'q' to quit): can you tell me more about yourself?
about
AI: I'm Pandora, your Personal Therapeutic AI Assistant. How are you feeling today
Prompt (press 'q' to quit): great, nice to meet you, can you help me
help
AI: Sure. Tell me how can i assist you
Prompt (press 'q' to quit): q


In [10]:
# def predict_intent(user_input):
#     user_input_vec = vectorizer.transform([user_input])
#     intent = model.predict(user_input_vec)[0]
#     return intent

# def generate_response(intent):
#     # print(intent)  # For debugging purposes, to see the intent detected
#     responses = {
#         'greeting': "Hello! How can I assist you today?",
#         'morning': "Good morning! How can I help you?",
#         'afternoon': "Good afternoon! What can I do for you?",
#         'evening': "Good evening! How can I assist you?",
#         'night': "Good night! Do you need any help before you sleep?",
#         'goodbye': "Goodbye! Have a great day!",
#         'thanks': "You're welcome! Is there anything else you need?",
#         'no-response': "I'm here whenever you need me.",
#         'neutral-response': "I understand. How can I help?",
#         'about': "I am an AI assistant here to help you with any questions you have.",
#         'skill': "I can assist you with information, advice, and support.",
#         'creation': "I was created by a team of developers to assist you.",
#         'name': "You can call me your AI assistant.",
#         'help': "I'm here to help. What do you need assistance with?",
#         'sad': "I'm sorry to hear that you're feeling sad. I'm here for you.",
#         'stressed': "It sounds like you're feeling stressed. How can I help you relax?",
#         'worthless': "I'm sorry you're feeling this way. Remember, you are valuable.",
#         'depressed': "It's important to talk to someone who can help. I'm here to listen.",
#         'happy': "I'm glad to hear you're happy! How can I assist you today?",
#         'casual': "What's on your mind?",
#         'anxious': "It sounds like you're feeling anxious. How can I support you?",
#         'not-talking': "It's okay if you don't feel like talking. I'm here when you're ready.",
#         'sleep': "Getting enough rest is important. Is there anything I can do to help?",
#         'scared': "I'm sorry you're feeling scared. Is there something specific worrying you?",
#         'death': "Talking about death can be difficult. It's important to share your feelings.",
#         'understand': "I'm here to understand and help you. What's on your mind?",
#         'done': "I'm here to assist with whatever you need. How can I help?",
#         'suicide': "If you're feeling like this, please reach out to someone who can help immediately.",
#         'hate-you': "I'm here to assist you, despite how you feel about me.",
#         'hate-me': "I'm sorry you feel that way. I'm here to help.",
#         'default': "I'm here to assist you with whatever you need.",
#         'jokes': "Why don't scientists trust atoms? Because they make up everything!",
#         'repeat': "Sure, I can repeat that for you.",
#         'wrong': "I'm sorry if I got something wrong. How can I correct it?",
#         'stupid': "I'm here to help with whatever you need.",
#         'location': "I'm here to help you find what you need.",
#         'something-else': "What else can I assist you with?",
#         'friends': "Having friends is important. How can I help you with your friendships?",
#         'ask': "What would you like to ask?",
#         'problem': "Let's see if we can solve this problem together.",
#         'no-approach': "I'm here to help you find a solution.",
#         'learn-more': "What would you like to learn more about?",
#         'user-agree': "I'm glad we agree. How can I assist you further?",
#         'meditation': "Meditation can be very helpful. Would you like some tips?",
#         'user-meditation': "It's great that you're meditating. How can I assist you?",
#         'pandora-useful': "I'm glad you find this useful. How can I help you more?",
#         'user-advice': "I'm here to give advice. What's on your mind?",
#         'learn-mental-health': "Learning about mental health is important. What would you like to know?",
#         'mental-health-fact': "Did you know that mental health is just as important as physical health?",
#     }
#     response = responses.get(intent, "I'm here to help with whatever you need.")
#     return response

# while True:
#     user_input = input()
#     intent = predict_intent(user_input)
#     response = generate_response(intent)
#     print("AI:", response)
